![ab_testing_image](ab_testing_image.jpg)

As a Data Scientist at a leading online travel agency, you’ve been tasked with evaluating the impact of a new search ranking algorithm designed to improve conversion rates. The Product team is considering a full rollout, but only if the experiment shows a clear positive effect on the conversion rate and does not lead to a longer time to book.

They have shared A/B test datasets with session-level booking data (`"sessions_data.csv"`) and user-level control/variant split (`"users_data.csv"`). Your job is to analyze and interpret the results to determine whether the new ranking system delivers a statistically significant improvement and provide a clear, data-driven recommendation.

## `sessions_data.csv`

| column | data type | description | 
|--------|-----------|-------------|
| `session_id` | `string` | Unique session identifier (unique for each row) |
| `user_id` | `string` | Unique user identifier (non logged-in users have missing user_id values; each user can have multiple sessions) |
| `session_start_timestamp` | `string` | When a session started |
| `booking_timestamp` | `string` | When a booking was made (missing if no booking was made during a session) |
| `time_to_booking` | `float` | time from start of the session to booking, in minutes (missing if no booking was made during a session) |
| `conversion` | `integer` | _New column to create:_ did session end up with a booking (0 if booking_timestamp or time_to_booking is Null, otherwise 1) |

<br>

## `users_data.csv`

| column | data type | description | 
|--------|-----------|-------------|
| `user_id` | `string` | Unique user identifier (only logged-in users in this table) |
| `experiment_group` | `string` | control / variant split for the experiment (expected to be equal 50/50) |

<br>

The full on criteria are the following:
- Primary metric (conversion) effect must be statistically significant and show positive effect (increase).
- Guardrail (time_to_booking) effect must either be statistically insignificant or show positive effect (decrease)

In [1]:
import pandas as pd
from scipy.stats import chisquare
from pingouin import ttest
from statsmodels.stats.proportion import proportions_ztest

In [2]:
#Reading datasets
sessions = pd.read_csv('sessions_data.csv')
users = pd.read_csv('users_data.csv')

In [3]:
#checking sample data
sessions.sample(5)

,session_id,user_id,session_start_timestamp,booking_timestamp,time_to_booking
6812,a1GykP6BpJ2mtH1m,9jhipk2G8s9n8nkt,2025-01-19 01:05:54.318530321,2025-01-19 01:19:35.816748357,13.691637
12331,hqZJ2ZYx7TvFwthh,NaN,2025-01-03 09:33:19.596735477,NaN,NaN
3413,VPpx4Yj8OPeR6Lj7,Ve4BnYvfZR4zOy2l,2025-01-24 12:43:00.801321268,NaN,NaN
15150,2X8NuBLGgitYSmyf,iHvi1DeJS7v2HyRo,2025-01-10 21:31:35.244395018,NaN,NaN
11063,C29BnnKwRBMzrY1W,h2ceMrxcGTmpyrTv,2025-01-22 12:14:07.436275005,NaN,NaN


In [4]:
users.sample(5)

,user_id,experiment_group
6080,mVy0glk8N5MeC8En,control
1889,0EDhn6OXW68eafUm,control
6047,O7lj3pxfAGhB5oUs,variant
7767,QGA056bZJ1EaBRUY,control
6474,oiWJEae55DThpNQS,variant


### Your solution

In [5]:
confidence_level = 0.90  # Set the pre-defined confidence level (90%)
alpha = 1 - confidence_level  # Significance level for hypothesis tests

In [6]:
#Define Functions
def estimate_effect_size(df: pd.DataFrame, metric: str) -> float:
    """
    Calculate realaive effect size

    Parameters:
    -df(pd.DataFrame):data with experiment_group('Control','Variant') and metric columns
    -metric(str): name of the metric column

    returns:
    Effect_size(float):average treatment effect(effect size)
    """
    avg_metric_per_group=df.groupby('experiment_group')[metric].mean()
    effect_size=avg_metric_per_group['variant']/avg_metric_per_group['control']-1
    return effect_size

In [7]:
sessions_x_users=pd.merge(sessions, users) #merging data sets

In [8]:
#compute primary metric(Conversion). Binary conversion flag: 1 if booking occured else 0
sessions_x_users['conversion']=sessions_x_users['booking_timestamp'].notnull().astype(int)
sessions_x_users.head(5)

,session_id,user_id,session_start_timestamp,booking_timestamp,time_to_booking,experiment_group,conversion
0,CP0lbAGnb5UNi3Ut,TcCIMrtQ75wHGXVj,2025-01-26 20:02:39.177358627,NaN,NaN,variant,0
1,UQAjrPYair63L1p8,TcCIMrtQ75wHGXVj,2025-01-20 16:12:51.536912203,NaN,NaN,variant,0
2,9zQrAPxV5oi2SzSa,TcCIMrtQ75wHGXVj,2025-01-28 03:46:40.839362144,NaN,NaN,variant,0
3,kkrz1M5vxrQ8wXRZ,GUGVzto9KGqeX3dc,2025-01-25 02:48:50.953303099,NaN,NaN,variant,0
4,ABZZFrwItZAPdYGP,v2EBIHmOdQfalI6k,2025-01-11 11:41:36.912253618,NaN,NaN,variant,0


In [9]:
#sample Ratio Mismatch Test
#Check if the number of users in each experiment group is balanced(a Basic A/A sanity check)
groups_count=sessions_x_users['experiment_group'].value_counts()
print(groups_count)

variant    7653
control    7630
Name: experiment_group, dtype: int64


In [10]:
n=sessions_x_users.shape[0] #total sample size
srm_chi2_stat, srm_chi2_pval=chisquare(f_obs=groups_count, f_exp=(n/2, n/2))
srm_chi2_pval=round(srm_chi2_pval,4) #pvalues
print(f'\nSRM\np-value:{srm_chi2_pval}') #if P < alpha, there's likely a sampling issue.



SRM
p-value:0.8524


In [11]:
#Effect Analysis - Primary Metric
#compute success counts and sample sizes for each group
success_counts=sessions_x_users.groupby('experiment_group')['conversion'].sum().loc[['control','variant']]
sample_sizes=sessions_x_users['experiment_group'].value_counts().loc[['control', 'variant']]

In [12]:
#run Z-test for proportions (binary conversion metric)
zstat_primary, pval_primary=proportions_ztest(
    success_counts,
    sample_sizes,
    alternative='two-sided',
)
pval_primary=round(pval_primary,4)

In [13]:
#Estimate effect size for the conversion metric
effect_size_primary=estimate_effect_size(sessions_x_users, 'conversion')
effect_size_primary=round(effect_size_primary,4)
print(f'\nPrimary metric\np-value: {pval_primary: .4f} | effect size :{effect_size_primary: .4f}')



Primary metric
p-value:  0.0002 | effect size : 0.1422


In [14]:
#Effect Analysis - GuardRail Metric
#T-test on time to booking for control vs variant
stats_guardrail=ttest(
    sessions_x_users.loc[(sessions_x_users['experiment_group']=='control'), 'time_to_booking'],
    sessions_x_users.loc[(sessions_x_users['experiment_group']=='variant'), 'time_to_booking'],
    alternative='two-sided',
)
pval_guardrail, tstat_guardrail=stats_guardrail['p-val'].values[0], stats_guardrail['T'].values[0]
pval_guardraiil=round(pval_guardrail,4)

In [17]:
#Estiamte effect Size for the guardrail metric
effect_size_guardrail=estimate_effect_size(sessions_x_users, 'time_to_booking')
effect_size_guardrail=round(effect_size_guardrail, 4)
print(f'\nGuardrail\np-value: {pval_guardrail} | effect size:  ')


Guardrail
p-value: 0.5364997062996586 | effect size:  


In [18]:
#Decision
#primary metric must be statistically significatn and show positive effect(INcrease)
criteria_full_on_primary=(pval_primary <alpha) & (effect_size_primary>0)

In [19]:
#Guardrail must be statistacially significant and show positive effect(decrease)
criteria_full_on_guardrail=(pval_guardrail>alpha) | (effect_size_guardrail<=0)

In [21]:
#fial launch decision based on both the metrics
if criteria_full_on_primary  and criteria_full_on_guardrail:
    decision_full_on='Yes'
    print('\nThe experiment results are significantly positive and the guardrail metric was not harmed, we are going full on!')
else:
    decision_full_on='No'
    print('\nThe experiment results are inconclusive or the guardrail metric was harmed, we are pulling back!')


The experiment results are significantly positive and the guardrail metric was not harmed, we are going full on!
